In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [16]:
# !%cd /content/drive/MyDrive/zeroshot_new_paper

In [114]:
# import sys
# sys.path.insert(0,'/content/drive/MyDrive/zeroshot_new_paper')

[autoreload of zeroberto failed: Traceback (most recent call last):
  File "/Users/alealcoforado/opt/anaconda3/envs/venv_zeroberto/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 261, in check
    superreload(m, reload, self.old_objects)
  File "/Users/alealcoforado/opt/anaconda3/envs/venv_zeroberto/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 459, in superreload
    module = reload(module)
  File "/Users/alealcoforado/opt/anaconda3/envs/venv_zeroberto/lib/python3.10/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 619, in _exec
  File "<frozen importlib._bootstrap_external>", line 879, in exec_module
  File "<frozen importlib._bootstrap_external>", line 1017, in get_code
  File "<frozen importlib._bootstrap_external>", line 947, in source_to_code
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "/Users/alealcoforado/Documents/Proj

# === KeyZeRoBERToFit v1.1.0 ===

# Dependencies

In [115]:
# !pip install transformers
# !pip install keybert==0.5.0
# !pip install datasets
# !pip install flair|
# %env PYTORCH_ENABLE_MPS_FALLBACK=1

In [116]:
# !pip install pandarallel

In [117]:
# !pip install evaluate

In [118]:
%load_ext autoreload
%autoreload 2
import nltk
import ipywidgets
import pandas as pd
import numpy as np
import time
from tqdm import tqdm
tqdm.pandas()
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)
from transformers import pipeline
import torch
import datetime
import datasets_handler
import evaluation_metrics
import os
import zeroberto
# def getAgora():
#     return str(datetime.datetime.now().strftime("%Y_%m_%d__%H_%M_%S"))


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


# Data Prep

## Source

In [120]:
which_dataset = 'folhauol' 
# which_dataset = 'bbc-news'

if which_dataset=='folhauol':
  arq = '/Users/alealcoforado/Documents/Projetos/Datasets/folhauol/folhauol_clean_df_articles.csv'
  # arq = '/content/drive/MyDrive/folhauol_clean_df_articles.csv'
  raw_data = pd.read_csv(arq)
  raw_data['full_text'] = raw_data['title'].astype(str)+raw_data['text'].astype(str)
  data_col = 'full_text'
  class_col = 'category'
  hyp_template = "O tema principal deste texto é {}."
  raw_data[class_col] = raw_data[class_col].map(datasets_handler.dict_classes_folha)
#raw_data.head(2)
if which_dataset == 'bbc-news':
  #o dataset bbc-news não tem coluna de título 
  # arq = '/content/drive/MyDrive/zeroshot_new_paper/bbc_news_train_plus_test.csv'
  arq = '/Users/alealcoforado/Documents/Projetos/Datasets/bbc-news/BBC News Train.csv'
  raw_data = pd.read_csv(arq, sep = ',')
  data_col = 'Text'
  class_col = 'Category'
  hyp_template = "this article is about {}."
# if which_dataset == 'sst2':
  ### trata dataset
  
raw_data.columns

Index(['Unnamed: 0', 'title', 'text', 'date', 'category', 'subcategory',
       'link', 'data', 'full_text'],
      dtype='object')

## Dataset build

### Train Data

In [121]:
# arq_train = '/content/drive/MyDrive/zeroshot_new_paper/BBC News Train.csv'
# train = pd.read_csv(arq_train)
train = raw_data
len(train)
# train = raw_data[~(raw_data['full_text'].isnull())].copy()
# train = raw_data[~(raw_data['full_text'].isnull())].sample(max_inferences,random_state=42).copy()

92241

In [122]:
labels_list = list(train[class_col].unique())

# Sentence Extraction

In [123]:
tokenizer = nltk.tokenize.punkt.PunktSentenceTokenizer()

In [124]:
try:
    print(train['sentences'])
except:
    train['sentences'] = train[data_col].progress_apply(lambda txt:tokenizer.tokenize(txt))
    train.head(2)

100%|██████████| 92241/92241 [00:17<00:00, 5350.67it/s]


In [125]:
train['len_sentences'] = train['sentences'].progress_apply(lambda txt:len(txt))
train[['len_sentences']].describe()

100%|██████████| 92241/92241 [00:00<00:00, 1949709.95it/s]


,len_sentences
count,92241.000000
mean,20.853709
std,18.024400
min,1.000000
25%,11.000000
50%,17.000000
75%,26.000000
max,1008.000000


# Zero-Shot

## Data

In [126]:
# zeroshot_method = "run_all"     #### not implemented yet
# max_inferences = len(train)#estava em 5000
max_inferences = 5000

# zeroshot_method = "probability_threshold"
probability_goal = 0.7

zeroshot_method = "top_n_goal"  #### not implemented yet
top_n_goal = 4

zeroshot_config = {
    'dataset':which_dataset,
    'method':zeroshot_method,
    'prob_goal':probability_goal,
    'top_n_goal':top_n_goal,
    'max_inferences':max_inferences,
    'labels':labels_list,#list(dict_classes_folha.values())
    'template': hyp_template,
    'random_state':422
}
zeroshot_config

{'dataset': 'folhauol',
 'method': 'top_n_goal',
 'prob_goal': 0.7,
 'top_n_goal': 4,
 'max_inferences': 5000,
 'labels': ['Poder e Política no Brasil',
  'Mercado',
  'Notícias de fora do Brasil',
  'Esporte',
  'Tecnologia',
  'Meio Ambiente',
  'Equilíbrio e Saúde',
  'Educação',
  'TV, Televisão e Entretenimento',
  'Ciência',
  'Turismo',
  'Comida'],
 'template': 'O tema principal deste texto é {}.',
 'random_state': 422}

In [127]:
sentences_to_classify = []
for sentences in train['sentences']:
  try:
    for sentence in sentences:
      sentences_to_classify.append(sentence)
      break
  except:
    print(train['sentences'])
train['sentence_to_zeroshot'] = sentences_to_classify
len(train)

92241

In [128]:
zeroshot_data = train.sample(zeroshot_config['max_inferences'],random_state=zeroshot_config['random_state'])
len(zeroshot_data)

5000

## Model

In [129]:
classifier = pipeline("zero-shot-classification",
                      model="joeddav/xlm-roberta-large-xnli",device="cpu") # device = 'cuda'

Some weights of the model checkpoint at joeddav/xlm-roberta-large-xnli were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Inference

In [130]:
%%time
zeroshot_results = zeroberto.runZeroShotPipeline(classifier,zeroshot_data['sentence_to_zeroshot'],zeroshot_config)
len(zeroshot_results)
### 100 predições ---> <240 segundos (4 min)
### 200 predições ---> <480 segundos (8 min)
### 1000 predições ---> <2400 segundos (40 min)
### 5000 predições ---> 12000 segundos (200 min) (3h 20min)

# data: 5000
{'dataset': 'folhauol', 'method': 'top_n_goal', 'prob_goal': 0.7, 'top_n_goal': 4, 'max_inferences': 5000, 'labels': ['Poder e Política no Brasil', 'Mercado', 'Notícias de fora do Brasil', 'Esporte', 'Tecnologia', 'Meio Ambiente', 'Equilíbrio e Saúde', 'Educação', 'TV, Televisão e Entretenimento', 'Ciência', 'Turismo', 'Comida'], 'template': 'O tema principal deste texto é {}.', 'random_state': 422}
Preds: 50  - Total time: 115.17 seconds
{'Poder e Política no Brasil': 0.0, 'Mercado': 0.0, 'Notícias de fora do Brasil': 0.0, 'Esporte': 0.0, 'Tecnologia': 0.0, 'Meio Ambiente': 0.0, 'Equilíbrio e Saúde': 0.0, 'Educação': 1.0, 'TV, Televisão e Entretenimento': 0.0, 'Ciência': 0.0, 'Turismo': 0.0, 'Comida': 0.0}
Preds: 100  - Total time: 224.87 seconds
{'Poder e Política no Brasil': 0.0, 'Mercado': 1.0, 'Notícias de fora do Brasil': 0.0, 'Esporte': 0.0, 'Tecnologia': 0.0, 'Meio Ambiente': 0.0, 'Equilíbrio e Saúde': 0.0, 'Educação': 1.0, 'TV, Televisão e Entretenimento': 0.0, 'C

In [ ]:
label_probabilities_df = zeroberto.formatZeroShotResults(zeroshot_results)

In [ ]:
def getTopLabel(row):
  return row.idxmax()
def getTopProb(row):
  return row.max()

label_results = label_probabilities_df.apply(getTopLabel,axis=1)
prob_results = label_probabilities_df.apply(getTopProb,axis=1)
label_results_df = pd.Series(label_results,name='prediction')

In [ ]:
true_labels_df = raw_data[raw_data.index.isin(train.index)][class_col]
final_result_df = pd.concat([true_labels_df,label_results_df],axis=1)

In [ ]:
vector_true = ( final_result_df[class_col].str.strip().str.lower() 
               == final_result_df['prediction'].str.strip().str.lower() )

## Metrics

In [ ]:
final_result_df_encoded = evaluation_metrics.Encoder(final_result_df,['prediction',class_col])

In [ ]:
final_result_df_encoded['predicted?'] = False
final_result_df_encoded.loc[0:len(zeroshot_results),'predicted?'] = True
#

In [ ]:
evaluation_metrics.get_metrics(final_result_df_encoded['prediction_code'][0:len(zeroshot_results)].to_list(),
                               final_result_df_encoded[class_col+'_code'][0:len(zeroshot_results)].to_list())

{'weighted': [{'accuracy': 0.20710387568217556},
  {'precision': 0.22067086421431711},
  {'recall': 0.20710387568217556},
  {'f1': 0.20003484316425052}],
 'macro': [{'accuracy': 0.20710387568217556},
  {'precision': 0.08420860432105003},
  {'recall': 0.0797967000425842},
  {'f1': 0.07557049338927573}]}

### Probabilities

In [ ]:
df_predictions_probs = pd.concat([final_result_df_encoded,
                                  pd.Series(prob_results,name='top_probability'),
                                  pd.Series(vector_true,name='right_wrong')],axis=1)

##### Top N results


In [ ]:
def get_top_n_results(dataframe_results,top_n=1):
    df_top_n = dataframe_results.sort_values(['top_probability','prediction'], ascending=False).groupby('prediction').head(top_n)

    accuracy_top_n = df_top_n.right_wrong.sum()/len(df_top_n.right_wrong)

    accuracy = { "accuracy in top {}".format(top_n) : round(accuracy_top_n,3)}
    print(accuracy)
    return 

get_top_n_results(df_predictions_probs,top_n = 1)
get_top_n_results(df_predictions_probs,top_n = 2)
get_top_n_results(df_predictions_probs,top_n = 3)
get_top_n_results(df_predictions_probs,top_n = 4)
get_top_n_results(df_predictions_probs,top_n = 5)
get_top_n_results(df_predictions_probs,top_n = 6)
get_top_n_results(df_predictions_probs,top_n = 7)
get_top_n_results(df_predictions_probs,top_n = 8)
get_top_n_results(df_predictions_probs,top_n = 9)
get_top_n_results(df_predictions_probs,top_n = 10)
get_top_n_results(df_predictions_probs,top_n = 11)
get_top_n_results(df_predictions_probs,top_n = 12)
get_top_n_results(df_predictions_probs,top_n = 13)
get_top_n_results(df_predictions_probs,top_n = 14)
get_top_n_results(df_predictions_probs,top_n = 15)
get_top_n_results(df_predictions_probs,top_n = 16)
get_top_n_results(df_predictions_probs,top_n = len(zeroshot_results))

{'accuracy in top 1': 0.25}
{'accuracy in top 2': 0.167}
{'accuracy in top 3': 0.139}
{'accuracy in top 4': 0.125}
{'accuracy in top 5': 0.117}
{'accuracy in top 6': 0.097}
{'accuracy in top 7': 0.095}
{'accuracy in top 8': 0.094}
{'accuracy in top 9': 0.102}
{'accuracy in top 10': 0.1}
{'accuracy in top 11': 0.106}
{'accuracy in top 12': 0.097}
{'accuracy in top 13': 0.096}
{'accuracy in top 14': 0.101}
{'accuracy in top 15': 0.094}
{'accuracy in top 16': 0.094}
{'accuracy in top 10811': 0.207}


## Save Results

In [ ]:
evaluation_metrics.saveZeroshotResults(zeroshot_config,df_predictions_probs)

predictions_and_probabilities_test_2023_02_10__19_42_09.csv
zeroshot_config_test_2023_02_10__19_42_09.csv


'2023_02_10__19_42_09'

# Load Previous Results

In [ ]:
# df_test_results = pd.read_csv(r'/Users/alealcoforado/Documents/Projetos/Datasets/predictions_and_probabilities_test1.csv')

In [ ]:
# df_test_results.index = df_test_results['Unnamed: 0']
# df_test_results.drop(columns="Unnamed: 0",inplace=True)